# **BA870 Group Project**

#### Cohort A Team 7:

#### Qiaoling Huang (qiaoling@bu.edu)

#### Shihan Li (shihanli@bu.edu)

#### Ziqin Ma (ziqinm@bu.edu)

#### Chenran Peng (cpeng3@bu.edu)

#### Elmira Ushirova (elu@bu.edu)


This is Team 7 from Cohort A. For this project, we would like to explore the impact of COVID-19 on **financial** performances of US companies. Specifically, we decided to focus on the **financial** **accounting** **characteristics**, using the financial ratios to explain the exposures to COVID-19 market shock.

# Data Preparation

Our preliminary data source comes from WRDS dataset and the Excel spread sheet provided. In addition, we used COVID-19 data for supplement analysis.

In [0]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

Saving BA870 Project Data 2020.csv to BA870 Project Data 2020.csv
Saving State_Level_COVID.csv to State_Level_COVID.csv
Saving WRDS.csv to WRDS.csv


## WRDS Data


In [0]:
df_w = pd.read_csv('WRDS.csv')
pd.DataFrame.from_records(df_w)
df_w.head()

,Global Company Key,Data Date,Data Year - Fiscal,Ticker Symbol,Company Name,Fiscal Year-end Month,Current Assets - Total,Assets - Total,Cash,Long-Term Debt - Total,Earnings Before Interest and Taxes,Employees,Current Liabilities - Total,"Property, Plant and Equipment - Total (Gross)",Sales/Turnover (Net),Interest and Related Expense - Total,GIC Industries,GIC Sub-Industries
0,1045,20191231,2019,AAL,AMERICAN AIRLINES GROUP INC,12,8206.00,59995.000,280.000,28875.00,3706.00,133.70,18311.000,62391.000,45768.000,1095.000,203020,20302010
1,1075,20191231,2019,PNW,PINNACLE WEST CAPITAL CORP,12,1030.03,18479.247,10.283,4884.43,671.96,6.21,2078.365,21402.118,3471.209,235.251,551010,55101010
2,1078,20191231,2019,ABT,ABBOTT LABORATORIES,12,15667.00,67887.000,3860.000,17416.00,5041.00,107.00,10863.000,17733.000,31904.000,670.000,351010,35101010
3,1161,20191231,2019,AMD,ADVANCED MICRO DEVICES,12,4597.00,6028.000,1466.000,685.00,583.00,11.40,2359.000,1473.000,6731.000,94.000,453010,45301020
4,1209,20190930,2019,APD,AIR PRODUCTS & CHEMICALS INC,9,4618.30,18942.800,2248.700,3227.40,2120.50,17.70,1820.900,22333.700,8918.900,150.500,151010,15101040


We extracted 1,361 unique ticker symbols form the provided spread sheet and imported them into WRDS. We selected the dat-to-day data from January 1, 2019 to December 31, 2019. We eyeballed the data and deletced the identifier columns, keeping only the ticker symbol column. We also deleted the `fiscal year` column, which was used for downloading purpose.

In [0]:
df_wr = df_w.loc[:,df_w.columns != 'Global Company Key']
df_wr = df_wr.loc[:,df_wr.columns != 'Data Date']
df_wr = df_wr.loc[:,df_wr.columns != 'Data Year - Fiscal']
df_wr = df_wr.loc[:,df_wr.columns != 'Company Name']
df_wr = df_wr.loc[:,df_wr.columns != 'Fiscal Year-end Month']
df_wr.head()

,Ticker Symbol,Current Assets - Total,Assets - Total,Cash,Long-Term Debt - Total,Earnings Before Interest and Taxes,Employees,Current Liabilities - Total,"Property, Plant and Equipment - Total (Gross)",Sales/Turnover (Net),Interest and Related Expense - Total,GIC Industries,GIC Sub-Industries
0,AAL,8206.00,59995.000,280.000,28875.00,3706.00,133.70,18311.000,62391.000,45768.000,1095.000,203020,20302010
1,PNW,1030.03,18479.247,10.283,4884.43,671.96,6.21,2078.365,21402.118,3471.209,235.251,551010,55101010
2,ABT,15667.00,67887.000,3860.000,17416.00,5041.00,107.00,10863.000,17733.000,31904.000,670.000,351010,35101010
3,AMD,4597.00,6028.000,1466.000,685.00,583.00,11.40,2359.000,1473.000,6731.000,94.000,453010,45301020
4,APD,4618.30,18942.800,2248.700,3227.40,2120.50,17.70,1820.900,22333.700,8918.900,150.500,151010,15101040


### Check Missing Values in Selected Columns

In [0]:
df_wr.isna().sum()

Ticker Symbol                                      0
Current Assets - Total                           252
Assets - Total                                     0
Cash                                              25
Long-Term Debt - Total                             8
Earnings Before Interest and Taxes                 0
Employees                                         33
Current Liabilities - Total                      252
Property, Plant and Equipment - Total (Gross)    212
Sales/Turnover (Net)                               0
Interest and Related Expense - Total             244
GIC Industries                                     0
GIC Sub-Industries                                 0
dtype: int64

We found some columns having missing values. As of now, we decided to keep those.

### Rename WRDS Columns

We renamed WRDS columns into financial abbreviation terms to make them more readable (and easy to code):

In [0]:
df_wr.columns = ['tic','CA','AT','Cash','LTDebt','EBIT','Employees','CL','PPE','Sales','INTX',
                 'GIC','GICsub']
df_wr.head()

,tic,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub
0,AAL,8206.00,59995.000,280.000,28875.00,3706.00,133.70,18311.000,62391.000,45768.000,1095.000,203020,20302010
1,PNW,1030.03,18479.247,10.283,4884.43,671.96,6.21,2078.365,21402.118,3471.209,235.251,551010,55101010
2,ABT,15667.00,67887.000,3860.000,17416.00,5041.00,107.00,10863.000,17733.000,31904.000,670.000,351010,35101010
3,AMD,4597.00,6028.000,1466.000,685.00,583.00,11.40,2359.000,1473.000,6731.000,94.000,453010,45301020
4,APD,4618.30,18942.800,2248.700,3227.40,2120.50,17.70,1820.900,22333.700,8918.900,150.500,151010,15101040


## Project Data



In [0]:
df_pro = pd.read_csv('BA870 Project Data 2020.csv')
pd.DataFrame.from_records(df_pro)
df_pro.head()


,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,Ticker,gvkey,datadate,fyear,cusip,conm_y,fyr,exchg,cik,fic,addzip,city,gsubind,naics,sic,state,PERMNO,date,NCUSIP,COMNAM,PERMCO,CUSIP
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,A,126554.0,20191031.0,2019.0,00846U101,AGILENT TECHNOLOGIES INC,10.0,11.0,1090872.0,USA,95051,Santa Clara,35203010.0,334516.0,3826.0,CA,87432.0,20191231.0,00846U10,AGILENT TECHNOLOGIES INC,36364.0,00846U10
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,AAL,1045.0,20191231.0,2019.0,02376R102,AMERICAN AIRLINES GROUP INC,12.0,14.0,6201.0,USA,76155,Fort Worth,20302010.0,481111.0,4512.0,TX,21020.0,20191231.0,02376R10,AMERICAN AIRLINES GROUP INC,20010.0,02376R10
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,AAON,21542.0,20191231.0,2019.0,360206,AAON INC,12.0,14.0,824142.0,USA,74107,Tulsa,20102010.0,333415.0,3585.0,OK,76868.0,20191231.0,36020,AAON INC,10817.0,36020
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,AAP,145977.0,20191231.0,2019.0,00751Y106,ADVANCE AUTO PARTS INC,12.0,11.0,1158449.0,USA,27604,Raleigh,25504050.0,441310.0,5531.0,NC,89217.0,20191231.0,00751Y10,ADVANCE AUTO PARTS INC,42402.0,00751Y10
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,AAPL,1690.0,20190930.0,2019.0,37833100,APPLE INC,9.0,14.0,320193.0,USA,95014,Cupertino,45202030.0,334220.0,3663.0,CA,14593.0,20191231.0,3783310,APPLE INC,7.0,3783310


### Select Variables from Project Data

We selected some useful columns from the spread sheet, including company tickers, company name, returns, and geographic information:

In [0]:
df_p = df_pro.iloc[:,np.r_[0:9,11,19:21,24,26]]
df_p.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0


### Check Missing Values in Selected Columns

In [0]:
df_p.isna().sum()

tic                0
conm_x             0
Last Price         0
YTD Ret            0
1-mth Ret          0
3-mth Ret          0
12-mth Ret         0
BothSP-NASDAQ      0
SP500              0
datadate          75
addzip            81
city              75
state            178
date              16
dtype: int64

We noticed that 178 companies have no State information. In the preview below, some companies are not located in the US. We keep them as of now.

In [0]:
df_p[df_p['state'].isnull()]

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date
14,ACGL,Arch Capital Grp Ltd,24.61,-0.4262,-0.4087,-0.4323,-0.2423,0,0,20191231.0,HM 08,Pembroke,NaN,20191231.0
19,ACN,Accenture Plc,152.15,-0.2774,-0.1650,-0.2748,-0.1413,0,1,20190831.0,2,Dublin,NaN,20191231.0
21,ACRX,Acelrx Pharmaceutica,1.05,-0.5024,-0.1985,-0.4853,-0.6948,0,0,NaN,NaN,NaN,NaN,20191231.0
25,ADMA,Adma Biologics,3.09,-0.2275,0.0300,-0.1782,-0.3965,0,0,NaN,NaN,NaN,NaN,20191231.0
26,ADMP,Adamis Pharmaceuticl,0.34,-0.5146,-0.3704,-0.5935,-0.8350,0,0,NaN,NaN,NaN,NaN,20191231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,XP,XP Inc Cl A,17.66,-0.5415,-0.5116,-0.5379,-0.4875,0,0,20191231.0,04551-065,Sao Paulo,NaN,20191231.0
1347,YNDX,Yandex N.V.,32.38,-0.2555,-0.1875,-0.2506,-0.0687,0,0,20191231.0,1118 BG,Schiphol,NaN,20191231.0
1349,YY,YY Inc,56.76,0.0752,0.0016,-0.0153,-0.3560,0,0,20191231.0,511442,Guangzhou,NaN,20191231.0
1353,ZG,Zillow Grp Inc Cl A,29.58,-0.3533,-0.4472,-0.3362,-0.1916,0,0,NaN,NaN,NaN,NaN,20191231.0


## Data Cleaning and Reformatting

Before calculating the ratios, we merged the WRDS data and project data together, called `df`:

In [0]:
df = pd.merge(df_p,df_wr, on = 'tic',how = 'left')
df.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,352030.0,35203010.0
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,203020.0,20302010.0
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,201020.0,20102010.0
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,255040.0,25504050.0
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,452020.0,45202030.0


With this master data frame, we checked the occurrances of NaN in each column. 

In [0]:
df.isna().sum()

tic                0
conm_x             0
Last Price         0
YTD Ret            0
1-mth Ret          0
3-mth Ret          0
12-mth Ret         0
BothSP-NASDAQ      0
SP500              0
datadate          75
addzip            81
city              75
state            178
date              16
CA               298
AT                46
Cash              71
LTDebt            54
EBIT              46
Employees         79
CL               298
PPE              258
Sales             46
INTX             290
GIC               46
GICsub            46
dtype: int64

We observed that there are 46 companies which do not have any information from WRDS:

In [0]:
df[df['GIC'].isnull()]

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub
26,ADMP,Adamis Pharmaceuticl,0.3400,-0.5146,-0.3704,-0.5935,-0.8350,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,ADUS,Addus Homecare Corp,71.5300,-0.2642,-0.0715,-0.2517,0.1187,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,ALRN,Aileron Therapeutics Inc,0.3133,-0.4532,-0.4669,-0.4822,-0.8210,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,AMPYW,Amplify Energy Corp WT Exp 05/04/2022,0.0071,-0.2900,-0.3107,-0.2900,-0.6450,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,BGNE,Beigene Ltd Ads,139.9800,-0.1555,-0.0947,-0.1377,-0.0324,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,BIDU,Baidu Inc,97.2000,-0.2310,-0.1761,-0.2735,-0.4443,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,BYSI,Beyondspring Inc,11.2100,-0.2768,-0.2436,-0.2566,-0.3283,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,CSIQ,Canadian Solar Inc,14.9000,-0.3258,-0.2874,-0.3165,-0.2215,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,DISCK,Discovery Comm Inc,16.8400,-0.4477,-0.3285,-0.4389,-0.3582,1,1,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,ECOR,Electrocore Inc,1.0300,-0.3522,0.4880,-0.3642,-0.8131,0,0,NaN,NaN,NaN,NaN,20191231.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 These 46 companies cannot add value to our analysis, so we decided to drop those 46 rows:

In [0]:
df.dropna(subset=['GIC'],inplace=True)
# Check the NaNs again
df.isna().sum()

tic                0
conm_x             0
Last Price         0
YTD Ret            0
1-mth Ret          0
3-mth Ret          0
12-mth Ret         0
BothSP-NASDAQ      0
SP500              0
datadate          29
addzip            35
city              29
state            132
date              15
CA               252
AT                 0
Cash              25
LTDebt             8
EBIT               0
Employees         33
CL               252
PPE              212
Sales              0
INTX             244
GIC                0
GICsub             0
dtype: int64

In [0]:
df.shape

(1315, 26)

Now we have a data frame with 1315 rows and 26 columns.

# Data to Financial Ratios

In the data preparation process, we have already observed some NaNs in the WRDS data as well as the given project data. Since the financial ratios are our primary variables, we decided to calculate the ratios first, and see the occurrances and distributions of NaNs. Once we have a glimpse of those missing values, we would be able to make decisions on how to impute them.

### Calculation Ratios: First Attempt

We chose 8 financial ratios as our input varaibles:

* Cash/Total_Assets

* Cash/Current_Liabilities

* Cash/Interest_Expense

* Current Ratio

* Long_Tern_Debt/Total_Assets

* EBIT/Interest_Expense

* Fixed_Assets/Total_Assets

* Net_Sales/Employee_Number

In [0]:
df['Cash/AT'] = df['Cash'] / df['AT']
df['Cash/CL'] = df['Cash'] / df['CL']
df['Cash/INTX'] = df['Cash'] / df['INTX']
df['CA/CL'] = df['CA'] / df['CL']
df['LTDebt/AT'] = df['LTDebt'] / df['AT']
df['EBIT/INTX'] = df['EBIT'] / df['INTX']
df['PPE/AT'] = df['PPE'] / df['AT']
df['Sales/Employees'] = df['Sales'] / df['Employees']
df.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,352030.0,35203010.0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,203020.0,20302010.0,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,201020.0,20102010.0,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,255040.0,25504050.0,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,452020.0,45202030.0,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292


Then we checked NaN, especially of the ratios:

In [0]:
df.isna().sum()

tic                  0
conm_x               0
Last Price           0
YTD Ret              0
1-mth Ret            0
3-mth Ret            0
12-mth Ret           0
BothSP-NASDAQ        0
SP500                0
datadate            29
addzip              35
city                29
state              132
date                15
CA                 252
AT                   0
Cash                25
LTDebt               8
EBIT                 0
Employees           33
CL                 252
PPE                212
Sales                0
INTX               244
GIC                  0
GICsub               0
Cash/AT             25
Cash/CL            271
Cash/INTX          269
CA/CL              252
LTDebt/AT            8
EBIT/INTX          244
PPE/AT             212
Sales/Employees     33
dtype: int64

We found that there are all ratio columns have missing values. The source of those missing might come from the raw data, because some companies might not have complete information. Directly dropping them is risky, and we do not want to lose information. Consequently, we decided to impute the missing value on the **raw** data, using the mean value of other companies in the **same** **industry**.

## NA Imputation

We extracted the first two digits of the GIC industry code from the WDRS data:

In [0]:
df["GIC"] = (df["GIC"]/10000).astype(int)
df.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,35,35203010.0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,20,20302010.0,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,20,20102010.0,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,25,25504050.0,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,45,45202030.0,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292


In the following chunks, we replaced the NaNs for each variable. Specifically, we calculated the mean of certain ratios for each industry (GIC) and saved that as an "average data frame". Then we joinined this data frame with `df` on industry code. With the merged data frame called `df1`, we replaced NaN in ratio column by the value in the average column, and repeated this process for 7 times:


* Current Asset:

In [0]:
# CA average calculation
CA_avg = df.groupby(['GIC'])['CA'].mean()
CA_avg_df = pd.DataFrame(CA_avg)
CA_avg_df.reset_index(inplace=True)
CA_avg_df.columns = ['GIC','CA_avg']
# Merge to a new data frame called `df1`
df1 = pd.merge(df, CA_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['CA'] = np.where(df1['CA'].isnull(), df1['CA_avg'], df1['CA'])

* Cash

In [0]:
# Cash average calculation
Cash_avg = df.groupby(['GIC'])['Cash'].mean()
Cash_avg_df = pd.DataFrame(Cash_avg)
Cash_avg_df.reset_index(inplace=True)
Cash_avg_df.columns = ['GIC','Cash_avg']
# Merge to `df1`
df1 = pd.merge(df1, Cash_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['Cash'] = np.where(df1['Cash'].isnull(), df1['Cash_avg'], df1['Cash'])

* Number of Employees

In [0]:
# Employee average calculation
Emp_avg = df.groupby(['GIC'])['Employees'].mean()
Emp_avg_df = pd.DataFrame(Emp_avg)
Emp_avg_df.reset_index(inplace=True)
Emp_avg_df.columns = ['GIC','Emp_avg']
# Merge to `df1`
df1 = pd.merge(df1, Emp_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['Employees'] = np.where(df1['Employees'].isnull(), df1['Emp_avg'], df1['Employees'])

* Long-Term Debt

In [0]:
# LTD average calculation
LTDebt_avg = df.groupby(['GIC'])['LTDebt'].mean()
LTDebt_avg_df = pd.DataFrame(LTDebt_avg)
LTDebt_avg_df.reset_index(inplace=True)
LTDebt_avg_df.columns = ['GIC','LTDebt_avg']
# Merge to `df1`
df1 = pd.merge(df1, LTDebt_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['LTDebt'] = np.where(df1['LTDebt'].isnull(), df1['LTDebt_avg'], df1['LTDebt'])

* Current Liability

In [0]:
# CL average calculation
CL_avg = df.groupby(['GIC'])['CL'].mean()
CL_avg_df = pd.DataFrame(CL_avg)
CL_avg_df.reset_index(inplace=True)
CL_avg_df.columns = ['GIC','CL_avg']
# Merge to `df1`
df1 = pd.merge(df1, CL_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['CL'] = np.where(df1['CL'].isnull(), df1['CL_avg'], df1['CL'])

* Property, Plant, and Equipment

In [0]:
# PPE average calculation
PPE_avg = df.groupby(['GIC'])['PPE'].mean()
PPE_avg_df = pd.DataFrame(PPE_avg)
PPE_avg_df.reset_index(inplace=True)
PPE_avg_df.columns = ['GIC','PPE_avg']
# Merge to `df1`
df1 = pd.merge(df1, PPE_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['PPE'] = np.where(df1['PPE'].isnull(), df1['PPE_avg'], df1['PPE'])

* Interests and Expenses

In [0]:
# INTX average calculation
INTX_avg = df.groupby(['GIC'])['INTX'].mean()
INTX_avg_df = pd.DataFrame(INTX_avg)
INTX_avg_df.reset_index(inplace=True)
INTX_avg_df.columns = ['GIC','INTX_avg']
# Merge to `df1`
df1 = pd.merge(df1, INTX_avg_df, on = 'GIC', how = 'left')
# Fill NaNs with the average
df1['INTX'] = np.where(df1['INTX'].isnull(), df1['INTX_avg'], df1['INTX'])

Finally, we double-checked the NaN. All raw data columns are complete:

In [0]:
df1.isna().sum()

tic                  0
conm_x               0
Last Price           0
YTD Ret              0
1-mth Ret            0
3-mth Ret            0
12-mth Ret           0
BothSP-NASDAQ        0
SP500                0
datadate            29
addzip              35
city                29
state              132
date                15
CA                   0
AT                   0
Cash                 0
LTDebt               0
EBIT                 0
Employees            0
CL                   0
PPE                  0
Sales                0
INTX                 0
GIC                  0
GICsub               0
Cash/AT             25
Cash/CL            271
Cash/INTX          269
CA/CL              252
LTDebt/AT            8
EBIT/INTX          244
PPE/AT             212
Sales/Employees     33
CA_avg               0
Cash_avg             0
Emp_avg              0
LTDebt_avg           0
CL_avg               0
PPE_avg              0
INTX_avg             0
dtype: int64

## Calculation Ratios: Second Attempt

With the updated data frame without missing values, we re-ran the process of calculating ratios:

In [0]:
df1['Cash/AT'] = df1['Cash'] / df1['AT']
df1['Cash/CL'] = df1['Cash'] / df1['CL']
df1['Cash/INTX'] = df1['Cash'] / df1['INTX']
df1['CA/CL'] = df1['CA'] / df1['CL']
df1['LTDebt/AT'] = df1['LTDebt'] / df1['AT']
df1['EBIT/INTX'] = df1['EBIT'] / df1['INTX']
df1['PPE/AT'] = df1['PPE'] / df1['AT']
df1['Sales/Employees'] = df1['Sales'] / df1['Employees']
df1.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,CA_avg,Cash_avg,Emp_avg,LTDebt_avg,CL_avg,PPE_avg,INTX_avg
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,35,35203010.0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466,2906.738092,868.147439,11.354120,2670.526684,2022.744752,1824.219882,120.344152
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,20,20302010.0,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624,3734.835543,1033.243299,36.032121,4369.181811,3185.170407,7583.812940,197.046739
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,20,20102010.0,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908,3734.835543,1033.243299,36.032121,4369.181811,3185.170407,7583.812940,197.046739
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,25,25504050.0,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493,5139.160822,1300.140341,58.930783,5284.773222,4492.999000,8671.324908,228.019184
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,45,45202030.0,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292,5267.066886,1359.992134,20.141654,3199.116250,3294.105397,3421.618870,143.502258


We checked the NaN again, and excited to find that there are no missing ratios. 

In [0]:
df1.isna().sum()

tic                  0
conm_x               0
Last Price           0
YTD Ret              0
1-mth Ret            0
3-mth Ret            0
12-mth Ret           0
BothSP-NASDAQ        0
SP500                0
datadate            29
addzip              35
city                29
state              132
date                15
CA                   0
AT                   0
Cash                 0
LTDebt               0
EBIT                 0
Employees            0
CL                   0
PPE                  0
Sales                0
INTX                 0
GIC                  0
GICsub               0
Cash/AT              0
Cash/CL              0
Cash/INTX            0
CA/CL                0
LTDebt/AT            0
EBIT/INTX            0
PPE/AT               0
Sales/Employees      0
CA_avg               0
Cash_avg             0
Emp_avg              0
LTDebt_avg           0
CL_avg               0
PPE_avg              0
INTX_avg             0
dtype: int64

## Variable Treatment



We would like to input a clean dataset into models, so we delected all columns used in calculating industry averages. 

In [0]:
# Delete average columns
df1 = df1.loc[:,df1.columns != 'CA_avg']
df1 = df1.loc[:,df1.columns != 'Cash_avg']
df1 = df1.loc[:,df1.columns != 'Emp_avg']
df1 = df1.loc[:,df1.columns != 'LTDebt_avg']
df1 = df1.loc[:,df1.columns != 'CL_avg']
df1 = df1.loc[:,df1.columns != 'PPE_avg']
df1 = df1.loc[:,df1.columns != 'INTX_avg']
df1.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,35,35203010.0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,20,20302010.0,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,20,20102010.0,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,25,25504050.0,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,45,45202030.0,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292


Then we selected ratios as X variables and saved as a data frame called `features`:

In [0]:
# Select ratios
features = df1.loc[:,'Cash/AT':'Sales/Employees']
features.head()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466
1,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624
2,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908
3,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493
4,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292


For safety, we checked the NaNs in `features`:

In [0]:
features.isna().sum()

Cash/AT            0
Cash/CL            0
Cash/INTX          0
CA/CL              0
LTDebt/AT          0
EBIT/INTX          0
PPE/AT             0
Sales/Employees    0
dtype: int64

### Winsorization

To get rid of the effect of outliers, we winsorized our X variables by 5% and 95% percentiles and printed out the sumamry of features:

In [0]:
# winsorize

from scipy import stats
for col in features.columns:
  features[col] = stats.mstats.winsorize(features[col], limits=[0.05,0.05], inplace=True) 

features.describe()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
count,1315.000000,1315.000000,1315.000000,1315.000000,1315.000000,1315.000000,1315.000000,1315.000000
mean,0.125128,0.840559,226.108843,2.463838,0.234693,5.898227,0.429847,536.205479
std,0.140628,1.119171,827.525703,2.301063,0.196004,31.087330,0.404597,536.116549
min,0.004200,0.010350,0.091810,0.657806,0.000302,-80.090909,0.018778,15.482480
25%,0.020446,0.101856,0.871593,1.062826,0.048948,0.263426,0.119430,204.290094
50%,0.070374,0.365103,4.251894,1.454577,0.211675,3.552659,0.270262,337.467333
75%,0.169279,1.062249,16.943635,2.846669,0.366896,9.414515,0.647474,652.692151
max,0.517227,4.160812,3744.565217,9.231352,0.669831,95.162050,1.398900,2145.728643


We checked the description and there is no NaN and Inf.

### Standardization

To better fit our features into the model, we standardized the ratios using minmaxscaler:

In [0]:
# Standardization

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(features)
feature1 = scaler.transform(features)

The output of `standardscaler()` is an array, and we need to convert output array `feature1` to a dataframe called `dataset`:

In [0]:
# Convert into a dataframe
dataset = pd.DataFrame(data=feature1[0:,1:],index=feature1[0:,0])

# Reset the index
dataset1 = dataset.reset_index()

# Adding column names 
dataset1.columns = ['Cash/AT','Cash/CL','Cash/INTX', 'CA/CL','LTDebt/AT','EBIT/INTX', 'PPE/AT', 
                    'Sales/Employees']

We examined the summary statistics:

In [0]:
dataset1.describe()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
count,1.315000e+03,1.315000e+03,1.315000e+03,1.315000e+03,1.315000e+03,1.315000e+03,1.315000e+03,1.315000e+03
mean,-8.848013e-17,8.645387e-17,3.381326e-17,9.682791e-17,-6.517811e-17,-4.399734e-17,-2.613034e-17,-1.788493e-16
std,1.000380e+00,1.000380e+00,1.000380e+00,1.000380e+00,1.000380e+00,1.000380e+00,1.000380e+00,1.000380e+00
min,-8.602390e-01,-7.420893e-01,-2.732278e-01,-7.851671e-01,-1.196305e+00,-2.767103e+00,-1.016384e+00,-9.716565e-01
25%,-7.446744e-01,-6.602961e-01,-2.722851e-01,-6.090861e-01,-9.480197e-01,-1.813261e-01,-7.675168e-01,-6.193460e-01
50%,-3.895042e-01,-4.249908e-01,-2.681987e-01,-4.387733e-01,-1.174790e-01,-7.547964e-02,-3.945809e-01,-3.708405e-01
75%,3.140739e-01,1.981596e-01,-2.528559e-01,1.664350e-01,6.747506e-01,1.131531e-01,5.380904e-01,2.173613e-01
max,2.789247e+00,2.967835e+00,4.253397e+00,2.942157e+00,2.220894e+00,2.872482e+00,2.396018e+00,3.003331e+00


Now we are prepared to run regressions!

# OLS Regression: ratios to stock returns

To see how predictive is the 2019 performance to stock returns of 2020 (up to April 3), we built an OLS regression model using `YTD_return` against the ratios we generated:

In [0]:
# OLS
import statsmodels.api as sm

X = dataset1
Y = df1['YTD Ret']
X = sm.add_constant(X) # adding a constant
model = sm.OLS(Y, X).fit()
print_model = model.summary()
print(print_model)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:                YTD Ret   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     10.64
Date:                Sat, 18 Apr 2020   Prob (F-statistic):           1.44e-14
Time:                        19:16:10   Log-Likelihood:                 42.623
No. Observations:                1315   AIC:                            -67.25
Df Residuals:                    1306   BIC:                            -20.61
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3130      0.006    -

In [0]:
## Save df1 to csv for graphs in Tableau
df1.to_excel('df1.xlsx')

## Explanation

Our first regression does not show promising results. We got a really low R-squared and adjusted R-squared: 6.1% and 5.5% accordingly. These results show that our 8 chosen ratios of companies' financial health at the end of 2019 explains roughly only 6% of total volatility of companies' earnings. However, it is also was somehow expected, because the overall situation on the market in the beginning of 2020 is way different from the market situation during 2019, because of COVID-19. Hence we cannot expect that information from 2019 can be any helpful, especially when at that time it was not known that COVID-19 would cause such a shock.

Nevertheless, there are some ratios in our regression that are statistically significant (p-value < 0.05): Cash/Total Assets, Cash/Current Liabilities, Current Assets/Current Liabilities, EBIT/Interest expense, PPE/Total Assets. Among these ratios, only **Cash/Total Assets, Current Assets/Current Liabilities, and EBIT/Interest expense** seem to positively correlate with 2020's earnings. In other words, the more Current Assets and Earnings before interest and Tax to cover their liabilities and expenses the company had, the higher their earnings in 2020 became. It follows our finance knowledge: in the emergency situations(right now), the liquid assets (cash and all current assets) usually plays the biggest role, due to the appearence of urgent unexpected expenses.

On the other hand, Cash/Current Liabilities and PPE/Total Assets are negatively correlated with 2020's earnings. For Cash/Current Liabilities, it means the more cash or less current liabilities the company had, the less earnings it got in 2020. This correlation is quite confusing, because as we saw with positively correlated ratios above, both increase in cash and decrease in current liabilities seemed to add only positive impact. However, for some reason this ratio has the opposite effect. For PPE/Total Assets, it means that the more fixed assets the company had at the end of 2019, the lower earnings it got in 2020. This ratio's correlation with 2020's earnings also makes sense, because in the emergency state conditions, lots of companies are suffering from lower sales and additional unexpected expenses, but they still have to pay all of the expenses related to fixed assets (such as rent etc.). So no wonder that right now companies with lots of fixed assets are facing higher expenses and in turn lower earnings.

Even though our regression overall does not explain most of the variation in 2020 earnings, still most of the statistically significant coefficients of ratios are seem to be in line with the current situation, i.e. most of them seem reasonable.



### Improvement

Based on the model summary of the previous regression, we realized that the adjusted R-square is quite low. One possible explanation of this low-variance explained is the influence Covid-19 shock. The pandemic occurred suddenly, and the 2019 ratios are hardly be able to capture that. We were wondering whether adding such information into the regression would help us increase the R-squared. To test our hypothesis, we added Covid-19 variables into our regression.


# OLS Regression: adding Covid-19 variables



## COVID-19 Data

Our COVID-19 data comes from the statistics of [Johns Hopkins Coronavirus Resource Center](https://github.com/CSSEGISandData/COVID-19). 
The repository includes time-series records of [confirmed cases](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv) and [deaths](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv) in the US. We selected the record of April 3 and demographic locations (State, City, and Population). 


In [0]:
df_covid = pd.read_csv('State_Level_COVID.csv')
pd.DataFrame.from_records(df_covid)
df_covid.head()

,state,Confirmed,population,Death,Death_1M,Confirmd_1M
0,AL,1495,4.903185,38,7.750064,304.903853
1,AK,157,0.731545,3,4.100910,214.614275
2,AS,0,0.055641,0,0.000000,0.000000
3,AZ,1937,7.278717,41,5.632861,266.118328
4,AR,704,3.017804,12,3.976401,233.282215


In [0]:
df_covid.shape

(56, 6)

Before importing the data to the evironment, we merged and changed the scale of the numbers. First, we merged the confirmed and death cases together by `state` and `city`. We also modified the `population` column by 1,000,000, making the unit of `population_M` as "million people" in each city. Then we calculated the number of confirmed and death per million people in each city, as `Confirmed_1M` and `Death_1M`.

Initially, we wanted to add these variables on city-level to get more precised calculation. However, when we joined our COVID data to the returns spreadsheet, we found that almost 900 cities were missing the COVID information. 

We have an intuition that the stock returns might be influenced by the govenment policy, which varied by State-by-State. The original data is built on city level, so we aggregated the State-level variables by each state, including 5 US territories (*American Samoa, Guam, Northern Mariana Islands, Puerto Rico, and Virgin Islands*). In addition, the raw data includes records from 2 cruise ships, Diamond Princess and Grand Princess, which do not have a stock return. We dropped the their records and imported to Colab.



## Merging Data

To avoid overwtirring, we made a copy of the master data frame `df1`, called `df2`:



In [0]:
df2 = df1

Before left-joining the COVID-19 data to the master data frame `df2`, we looked at the 132 companies with NaN in `state` column:

In [0]:
df2.isna().sum()

tic                  0
conm_x               0
Last Price           0
YTD Ret              0
1-mth Ret            0
3-mth Ret            0
12-mth Ret           0
BothSP-NASDAQ        0
SP500                0
datadate            29
addzip              35
city                29
state              132
date                15
CA                   0
AT                   0
Cash                 0
LTDebt               0
EBIT                 0
Employees            0
CL                   0
PPE                  0
Sales                0
INTX                 0
GIC                  0
GICsub               0
Cash/AT              0
Cash/CL              0
Cash/INTX            0
CA/CL                0
LTDebt/AT            0
EBIT/INTX            0
PPE/AT               0
Sales/Employees      0
dtype: int64

In [0]:
df2[df2['state'].isnull()]

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees
14,ACGL,Arch Capital Grp Ltd,24.61,-0.4262,-0.4087,-0.4323,-0.2423,0,0,20191231.0,HM 08,Pembroke,NaN,20191231.0,8468.693048,37885.361,726.230,2506.432,1882.841,4.300000,7968.092524,5561.605653,6806.006,120.872000,40,40301040.0,0.019169,0.091142,6.008257,1.062826,0.066158,15.577148,0.146801,1582.792093
19,ACN,Accenture Plc,152.15,-0.2774,-0.1650,-0.2748,-0.1413,0,1,20190831.0,2,Dublin,NaN,20191231.0,15450.601000,29789.880,6126.694,16.247,6305.074,492.000000,11061.896000,3347.195000,43215.013,22.963000,45,45102010.0,0.205664,0.553856,266.807212,1.396741,0.000545,274.575360,0.112360,87.835392
21,ACRX,Acelrx Pharmaceutica,1.05,-0.5024,-0.1985,-0.4853,-0.6948,0,0,NaN,NaN,NaN,NaN,20191231.0,71.688000,91.356,14.684,115.840,-54.205,0.099000,13.611000,28.032000,2.289,0.998000,35,35202010.0,0.160734,1.078833,14.713427,5.266916,1.268006,-54.313627,0.306844,23.121212
25,ADMA,Adma Biologics,3.09,-0.2275,0.0300,-0.1782,-0.3965,0,0,NaN,NaN,NaN,NaN,20191231.0,85.820000,127.091,26.752,84.501,-41.424,0.314000,14.028000,37.466000,29.349,8.993000,35,35201010.0,0.210495,1.907043,2.974758,6.117764,0.664886,-4.606249,0.294797,93.468153
39,AFYA,Afya Ltd Cl A,18.10,-0.3326,-0.2797,-0.3382,-0.2487,0,0,NaN,NaN,NaN,NaN,20191231.0,276.584000,724.557,234.651,66.815,51.051,58.930783,82.900000,112.095000,186.741,15.868000,25,25302010.0,0.323854,2.830531,14.787686,3.336357,0.092215,3.217230,0.154708,3.168819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,WMGI,Wright Med Grp NV,26.32,-0.1365,-0.1285,-0.1399,-0.1351,0,0,20191231.0,1097 JB,Amsterdam,NaN,20191231.0,743.658000,2585.640,166.856,751.314,24.553,3.030000,850.008000,668.312000,920.900,83.449000,35,35101010.0,0.064532,0.196299,1.999497,0.874884,0.290572,0.294228,0.258471,303.927393
1297,XP,XP Inc Cl A,17.66,-0.5415,-0.5116,-0.5379,-0.4875,0,0,20191231.0,04551-065,Sao Paulo,NaN,20191231.0,8468.693048,10852.429,16.354,366.381,366.987,14.597730,7968.092524,5561.605653,1275.691,20.997000,40,40203020.0,0.001507,0.002052,0.778873,1.062826,0.033760,17.478068,0.512476,87.389684
1302,YNDX,Yandex N.V.,32.38,-0.2555,-0.1875,-0.2506,-0.0687,0,0,20191231.0,1118 BG,Schiphol,NaN,20191231.0,1937.340000,4688.986,908.641,174.609,424.306,10.092000,749.591000,24943.013929,2824.914,1.192000,50,50203010.0,0.193782,1.212182,762.282718,2.584529,0.037238,355.961409,5.319490,279.916171
1304,YY,YY Inc,56.76,0.0752,0.0016,-0.0153,-0.3560,0,0,20191231.0,511442,Guangzhou,NaN,20191231.0,4136.142000,7498.128,559.175,742.690,95.239,30.744429,1096.886000,24943.013929,3673.157,38.276000,50,50203010.0,0.074575,0.509784,14.609024,3.770804,0.099050,2.488217,3.326566,119.473907


We observed that those companies do not have a US address or even missing in `City`. Since our focus is the COVID-19 impact in the US, we decided to drop those 132 companies for this regression:

In [0]:
# Drop 132 rows with missing state
df2.dropna(subset = ['state'],inplace = True)
df2.isna().sum()

tic                 0
conm_x              0
Last Price          0
YTD Ret             0
1-mth Ret           0
3-mth Ret           0
12-mth Ret          0
BothSP-NASDAQ       0
SP500               0
datadate            0
addzip              0
city                0
state               0
date               13
CA                  0
AT                  0
Cash                0
LTDebt              0
EBIT                0
Employees           0
CL                  0
PPE                 0
Sales               0
INTX                0
GIC                 0
GICsub              0
Cash/AT             0
Cash/CL             0
Cash/INTX           0
CA/CL               0
LTDebt/AT           0
EBIT/INTX           0
PPE/AT              0
Sales/Employees     0
dtype: int64

We then looked at the unique States listed:

In [0]:
print(len(df2.state.unique()))
df2.state.unique()

51


array(['CA', 'TX', 'OK', 'NC', 'NY', 'AZ', 'IL', 'PA', 'GA', 'MA', 'TN',
       'FL', 'NJ', 'WA', 'OH', 'MO', 'CO', 'VA', 'MD', 'NV', 'MI', 'LA',
       'MN', 'NE', 'IN', 'WI', 'CT', 'KY', 'AR', 'SC', 'PR', 'ME', 'MS',
       'SD', 'IA', 'DC', 'KS', 'RI', 'WV', 'NH', 'OR', 'DE', 'VT', 'ID',
       'UT', 'HI', 'MT', 'ON', 'QC', 'BC', 'AL'], dtype=object)

We noticed that some States seem not in the US. To be precise, we compared the State codes above to the [full list](https://www.factmonster.com/us/postal-information/state-abbreviations-and-state-postal-codes) of US State abbrevations, then filterd out 3 States. we checked the `city` column for each of them:

In [0]:
# 1. BC
df2[df2.state == 'BC'].city

709     Vancouver
739     Vancouver
886     Vancouver
1152      Nanaimo
Name: city, dtype: object

We searched online and knew that Vancouver and Nnaimo are cities in British Columbia, Canada;

In [0]:
# 2. ON
df2[df2.state == 'ON'].city

470     Toronto
1129    Toronto
Name: city, dtype: object

Toronto is in Ontario, Canada;

In [0]:
# 3. QC
df2[df2.state == 'QC'].city

700    Terrebonne
Name: city, dtype: object

Terrebonne is in Québec, Canada.

All these companies are located in Canada. Since our focus for this part is on the COVID-19 impact within the US, we decided to drop those Canadian companies. The new data frame is called `df3`:

In [0]:
df3 = df2[df2.state!='BC']
df3 = df3[df3.state!='ON']
df3 = df3[df3.state!='QC']

Then we merged the Covid-19 data to the cleaned `df3`, named as `df4`:

In [0]:
df4 = pd.merge(df3,df_covid, on = 'state', how = 'left')
df4.head()

,tic,conm_x,Last Price,YTD Ret,1-mth Ret,3-mth Ret,12-mth Ret,BothSP-NASDAQ,SP500,datadate,addzip,city,state,date,CA,AT,Cash,LTDebt,EBIT,Employees,CL,PPE,Sales,INTX,GIC,GICsub,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
0,A,Agilent Technologies,70.42,-0.1745,-0.1233,-0.1673,-0.1406,0,1,20191031.0,95051,Santa Clara,CA,20191231.0,3189.000,9452.000,1382.000,1791.000,1035.000,16.30,2080.000,1775.000,5163.000,74.000,35,35203010.0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466,12004,39.512223,265,6.706785,303.804724
1,AAL,American Airlines Gp,9.39,-0.6726,-0.4739,-0.6604,-0.7214,1,1,20191231.0,76155,Fort Worth,TX,20191231.0,8206.000,59995.000,280.000,28875.000,3706.000,133.70,18311.000,62391.000,45768.000,1095.000,20,20302010.0,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624,5734,28.995881,100,3.448766,197.752226
2,AAON,Aaon Inc,44.91,-0.0911,-0.2006,-0.1080,-0.0194,0,0,20191231.0,74107,Tulsa,OK,20191231.0,187.549,371.424,26.797,0.000,67.348,2.29,56.028,359.019,469.333,0.000,20,20102010.0,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908,990,3.956971,38,9.603305,250.191371
3,AAP,Advance Auto Parts Inc,84.65,-0.4715,-0.3505,-0.4690,-0.5070,0,1,20191231.0,27604,Raleigh,NC,20191231.0,5695.543,11248.525,418.665,2764.479,767.493,67.00,4477.702,5836.387,9709.003,39.898,25,25504050.0,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493,2251,10.488084,27,2.574350,214.624521
4,AAPL,Apple Inc,241.41,-0.1779,-0.1656,-0.1883,0.2358,1,1,20190930.0,95014,Cupertino,CA,20191231.0,162819.000,338516.000,48844.000,91807.000,63930.000,137.00,105718.000,95957.000,260174.000,3576.000,45,45202030.0,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292,12004,39.512223,265,6.706785,303.804724


Always check NaN:

In [0]:
df4.isna().sum()

tic                 0
conm_x              0
Last Price          0
YTD Ret             0
1-mth Ret           0
3-mth Ret           0
12-mth Ret          0
BothSP-NASDAQ       0
SP500               0
datadate            0
addzip              0
city                0
state               0
date               13
CA                  0
AT                  0
Cash                0
LTDebt              0
EBIT                0
Employees           0
CL                  0
PPE                 0
Sales               0
INTX                0
GIC                 0
GICsub              0
Cash/AT             0
Cash/CL             0
Cash/INTX           0
CA/CL               0
LTDebt/AT           0
EBIT/INTX           0
PPE/AT              0
Sales/Employees     0
Confirmed           0
population          0
Death               0
Death_1M            0
Confirmd_1M         0
dtype: int64

In [0]:
# df4.to_excel("df4.xlsx")

## Variable Treatment

We selected features to a new data frame, `feature2`, and did variable treatment on that:

In [0]:
feature2 = df4.iloc[:,np.r_[26:34,37:39]]
feature2.head()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Death_1M,Confirmd_1M
0,0.146212,0.664423,18.675676,1.533173,0.189484,13.986486,0.187791,316.748466,6.706785,303.804724
1,0.004667,0.015291,0.255708,0.448146,0.481290,3.384475,1.039937,342.318624,3.448766,197.752226
2,0.072147,0.478279,inf,3.347416,0.000000,inf,0.966602,204.948908,9.603305,250.191371
3,0.037220,0.093500,10.493383,1.271979,0.245764,19.236378,0.518858,144.910493,2.574350,214.624521
4,0.144289,0.462022,13.658837,1.540126,0.271204,17.877517,0.283464,1899.080292,6.706785,303.804724


### Winsorization

We winsorized on `feature2` at 5% and 95%: 

In [0]:
from scipy import stats
for col in feature2.columns:
  feature2[col] = stats.mstats.winsorize(feature2[col], limits=[0.05,0.05], inplace=True) 

feature2.describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Death_1M,Confirmd_1M
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000
mean,0.117211,0.797594,115.807726,2.341242,0.238153,6.679472,0.422024,564.581768,27.316615,989.354482
std,0.131311,1.075996,390.969800,2.141564,0.196513,24.551768,0.382468,559.189199,42.414151,1246.211135
min,0.003986,0.009751,0.088959,0.643500,0.000361,-54.496878,0.021075,34.426654,3.448766,197.752226
25%,0.018741,0.092799,0.815556,1.062826,0.051792,0.316621,0.121038,220.228942,6.672427,283.340890
50%,0.066516,0.346831,4.004798,1.420299,0.218791,3.717887,0.273583,353.186539,7.785030,449.102629
75%,0.161913,1.022729,15.672360,2.707194,0.369244,9.419185,0.648451,683.934012,27.856353,1276.077148
max,0.473225,4.044864,1730.753049,8.678015,0.673933,78.215909,1.285276,2273.046832,152.107250,4358.672621


We checked the description and there is no NaN and Inf.

### Standardization

We standardized the ratios using minmaxscaler:

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(feature2)
feature3 = scaler.transform(feature2)

# Convert to a data frame
dataset_c = pd.DataFrame(data=feature3[0:,1:],index=feature3[0:,0])
# Reset the index
dataset_c = dataset_c.reset_index()
# Renamed the columns
dataset_c.columns = ['Cash/AT','Cash/CL','Cash/INTX', 'CA/CL','LTDebt/AT','EBIT/INTX', 'PPE/AT', 'Sales/Employees', 'Death_1M','Confirmd_1M']

We examined the summary statistics:

In [0]:
dataset_c.describe()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Death_1M,Confirmd_1M
count,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03
mean,9.818299e-18,-1.755965e-17,-2.666990e-17,1.307769e-16,4.795861e-17,-4.546864e-17,1.169935e-16,-3.586275e-17,-1.586033e-17,-9.421791e-17
std,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00
min,-8.626284e-01,-7.325108e-01,-2.961047e-01,-7.930955e-01,-1.210574e+00,-2.492789e+00,-1.048765e+00,-9.484816e-01,-5.629726e-01,-6.354774e-01
25%,-7.502182e-01,-6.552949e-01,-2.942455e-01,-5.972086e-01,-9.487417e-01,-2.592709e-01,-7.872899e-01,-6.160694e-01,-4.869358e-01,-5.667691e-01
50%,-3.862336e-01,-4.191042e-01,-2.860847e-01,-4.302162e-01,-9.856875e-02,-1.206774e-01,-3.882759e-01,-3.781997e-01,-4.606928e-01,-4.336999e-01
75%,3.405745e-01,2.093235e-01,-2.562294e-01,1.709532e-01,6.673715e-01,1.116367e-01,5.922672e-01,2.135288e-01,1.273086e-02,2.301734e-01
max,2.712387e+00,3.019206e+00,4.132371e+00,2.960204e+00,2.218509e+00,2.914938e+00,2.258016e+00,3.056554e+00,2.943445e+00,2.704800e+00


## Regression

Now, we would like to run an OLS regression with COVID-19 factors, confirmed per million people and death per million people:

In [0]:
# Adjusted OLS
import statsmodels.api as sm

X_c = dataset_c
Y_c = df4['YTD Ret']
X_c = sm.add_constant(X_c) # adding a constant
model_c = sm.OLS(Y_c, X_c).fit()
print_model_c = model_c.summary()
print(print_model_c)

                            OLS Regression Results                            
Dep. Variable:                YTD Ret   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     8.423
Date:                Sat, 18 Apr 2020   Prob (F-statistic):           2.44e-13
Time:                        19:35:17   Log-Likelihood:                 28.765
No. Observations:                1176   AIC:                            -35.53
Df Residuals:                    1165   BIC:                             20.24
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3164      0.007    -

### Explanation

In this regression, we see slight improvements in R-squared and adjusted R-squared: now it is 6.7% and 5.9% accordingly. Though very tiny, the model was still improved. In terms of ratios, the p-values and coefficients did change slightly, but overall results are similar with the previous regression. 

Unfortunately, our new two variables are not statistically significant: both of these variables have p-value over 0.05. The coefficient for death rate per 1M population on state level is negative, which means that the more deaths from COVID-19, the less earnings companies get. We can explain with the simple fact that the more people in a state, the more sales the companies in that state would probably get. 

However, the coefficient for number of confirmed cases per 1M population on the state level is positive, which can be interpreted as the more number of cases in a state, the more earnings companies in that state get. This correlation is questionable. One way we may think of the reason behind that as those states with higher confirmed cases have stricter stay at home orders, which may make people buy more things.

Overall, even though the additional two variables are not statistically significant, their presence still improved the R-squared a little bit.

# Enhanced Regression

We decided to add the original COVID-19 variables (Comfirmed Cases, Death numbers, and population) into our regression, started with a new feature data frame (`feature_new`):

In [0]:
feature_new = df4.iloc[:,np.r_[26:39]]
feature_new.head()

## Variable Treatment

### Winsorization

We winsorized on `feature_new` at 5% and 95%: 

In [0]:
from scipy import stats
for col in feature_new.columns:
  feature_new[col] = stats.mstats.winsorize(feature_new[col], limits=[0.05,0.05], inplace=True) 

feature_new.describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000
mean,0.117211,0.797594,115.807726,2.341242,0.238153,6.679472,0.422024,564.581768,16283.993197,17.292074,478.786565,27.316615,989.354482
std,0.131311,1.075996,390.969800,2.141564,0.196513,24.551768,0.382468,559.189199,27893.098002,13.161514,989.993502,42.414151,1246.211135
min,0.003986,0.009751,0.088959,0.643500,0.000361,-54.496878,0.021075,34.426654,770.000000,3.017804,14.000000,3.448766,197.752226
25%,0.018741,0.092799,0.815556,1.062826,0.051792,0.316621,0.121038,220.228942,3067.000000,6.892503,51.000000,6.672427,283.340890
50%,0.066516,0.346831,4.004798,1.420299,0.218791,3.717887,0.273583,353.186539,8904.000000,10.617423,192.000000,7.785030,449.102629
75%,0.161913,1.022729,15.672360,2.707194,0.369244,9.419185,0.648451,683.934012,12004.000000,28.995881,265.000000,27.856353,1276.077148
max,0.473225,4.044864,1730.753049,8.678015,0.673933,78.215909,1.285276,2273.046832,102987.000000,39.512223,3594.000000,152.107250,4358.672621


We checked the description and there is no NaN and Inf.

### Standardization

We standardized the ratios using minmaxscaler:

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(feature_new)
feature_new1 = scaler.transform(feature_new)

# Convert to a data frame
dataset_c1 = pd.DataFrame(data=feature_new1[0:,1:],index=feature_new1[0:,0])
# Reset the index
dataset_c1 = dataset_c1.reset_index()
# Rename columns
dataset_c1.columns = ['Cash/AT','Cash/CL','Cash/INTX', 'CA/CL','LTDebt/AT','EBIT/INTX', 'PPE/AT', 
                      'Sales/Employees', 'Confirmed','population','Death','Death_1M','Confirmd_1M']

Summary statistics:

In [0]:
dataset_c1.describe()

,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
count,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03,1.176000e+03
mean,9.818299e-18,-1.755965e-17,-2.666990e-17,1.307769e-16,4.795861e-17,-4.546864e-17,1.169935e-16,-3.586275e-17,1.935338e-17,1.352848e-16,2.058066e-17,-1.586033e-17,-9.421791e-17
std,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00,1.000425e+00
min,-8.626284e-01,-7.325108e-01,-2.961047e-01,-7.930955e-01,-1.210574e+00,-2.492789e+00,-1.048765e+00,-9.484816e-01,-5.564313e-01,-1.085008e+00,-4.696842e-01,-5.629726e-01,-6.354774e-01
25%,-7.502182e-01,-6.552949e-01,-2.942455e-01,-5.972086e-01,-9.487417e-01,-2.592709e-01,-7.872899e-01,-6.160694e-01,-4.740462e-01,-7.904862e-01,-4.322943e-01,-4.869358e-01,-5.667691e-01
50%,-3.862336e-01,-4.191042e-01,-2.860847e-01,-4.302162e-01,-9.856875e-02,-1.206774e-01,-3.882759e-01,-3.781997e-01,-2.646939e-01,-5.073497e-01,-2.898085e-01,-4.606928e-01,-4.336999e-01
75%,3.405745e-01,2.093235e-01,-2.562294e-01,1.709532e-01,6.673715e-01,1.116367e-01,5.922672e-01,2.135288e-01,-1.535080e-01,8.896231e-01,-2.160393e-01,1.273086e-02,2.301734e-01
max,2.712387e+00,3.019206e+00,4.132371e+00,2.960204e+00,2.218509e+00,2.914938e+00,2.258016e+00,3.056554e+00,3.109726e+00,1.688985e+00,3.148040e+00,2.943445e+00,2.704800e+00


## Regression

In [0]:
import statsmodels.api as sm

X_c1 = dataset_c1
Y_c1 = df4['YTD Ret']
X_c1 = sm.add_constant(X_c1) # adding a constant
model_c1 = sm.OLS(Y_c1, X_c1).fit()
print_model_c1 = model_c1.summary()
print(print_model_c1)

                            OLS Regression Results                            
Dep. Variable:                YTD Ret   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     8.562
Date:                Sat, 18 Apr 2020   Prob (F-statistic):           7.92e-17
Time:                        19:55:58   Log-Likelihood:                 41.507
No. Observations:                1176   AIC:                            -55.01
Df Residuals:                    1162   BIC:                             15.96
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3164      0.007    -

### Explanation

After adding three more variables, we can see the overall adjusted R square has improved from 0.059 to 0.077. Variables that are significant in the previous regression keep their performance, except `Cash/AT`, which becomes insignificant.

In terms of the added variables, `population` has a positive coefficient and is significant. We believed that the more population in the area the better economy there. Therefore companies can generate more returns in a large population area. Both `Confirmed` and `Death` are not significant, but we can get a sense of the negative coefficients: if the area has more confirmed and more death, companies might be impacted more severely, having less returns. 

The coefficient of `Death_1M` is positive, saying that every 1 unit increase in death per million people can increase the stock return by 0.067. If we followed the same logic, which explains the positive coefficient of `Confirmed_1M` in the second regression, it is likely that the increase in death per million population warns residents in that state, so more people will choose or required to stay at home. A more intensive quaritine rule might make people purchase and store more things. But this approach is still not valid enough, because is is not significant.

# State-level Regression: with all variables

In previous 2 regressions with COVID-19 data, we matched the State-level information with each company. One potential problem for this approach is that the situation in the same state can vary by county and city. For companies in one State, adding uniform variables to them might not help in prediction. Thus, we aggregated our features and stock returns by state to see the regression performance on State-level.


## Re-build Dataframe

We first concatenated the feartues from Enhanced Regression (already treated) with the `YTD Ret` and `state` from `df4` to a new data frame called `df5A`:

In [0]:
df5A = pd.concat([df4[['YTD Ret', 'state']], dataset_c1], axis=1, ignore_index=True)
# Rename the columns
df5A.columns = ['YTD Ret','state','Cash/AT','Cash/CL','Cash/INTX', 'CA/CL','LTDebt/AT','EBIT/INTX', 'PPE/AT', 
                'Sales/Employees','Confirmed','population','Death','Death_1M','Confirmd_1M']
df5A.head()

,YTD Ret,state,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
0,-0.1745,CA,0.220957,-0.123818,-0.248544,-0.377487,-0.247770,0.297743,-0.612684,-0.443390,-0.153508,1.688985,-0.216039,-0.486125,-0.550341
1,-0.6726,TX,-0.857443,-0.727359,-0.295678,-0.793096,1.237785,-0.134263,1.616280,-0.397643,-0.378390,0.889623,-0.382778,-0.562973,-0.635477
2,-0.0911,OK,-0.343332,-0.296889,4.132371,0.470031,-1.210574,2.914938,1.424456,-0.643406,-0.548541,-1.013620,-0.445431,-0.417805,-0.593381
3,-0.4715,NC,-0.609433,-0.654643,-0.269482,-0.499503,0.038745,0.511664,0.253291,-0.750819,-0.503313,-0.517181,-0.456547,-0.562973,-0.621933
4,-0.1779,CA,0.206299,-0.312004,-0.261382,-0.374239,0.168261,0.456293,-0.362432,2.387504,-0.153508,1.688985,-0.216039,-0.486125,-0.550341


Then we aggregated the State-level variables for each State into `df5B`:

In [0]:
df5B = df5A.groupby('state', as_index=False).agg({'YTD Ret': 'mean', 'Cash/AT': 'mean', 'Cash/CL': 'mean', 'Cash/INTX': 'mean', 
                                                  'CA/CL': 'mean', 'LTDebt/AT': 'mean', 'LTDebt/AT':'mean', 'EBIT/INTX': 'mean',
                                                  'PPE/AT': 'mean', 'Sales/Employees': 'mean', 'Confirmed': 'mean', 'population': 'mean', 
                                                  'Death': 'mean', 'Death_1M': 'mean', 'Confirmd_1M': 'mean'})
df5B.head()

,state,YTD Ret,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
0,AL,-0.374000,-0.773769,-0.519199,-0.292916,-0.361666,-0.603718,-0.132290,0.226263,0.006105,-0.530428,-0.941697,-0.445431,-0.461518,-0.549459
1,AR,-0.327512,-0.641639,-0.635215,-0.289180,-0.560611,-0.123214,-0.069757,0.707731,-0.240831,-0.556431,-1.085008,-0.469684,-0.550527,-0.606955
2,AZ,-0.367155,0.058631,-0.082100,-0.044275,-0.112756,0.238629,-0.151101,0.174516,-0.337302,-0.514575,-0.761130,-0.442400,-0.511456,-0.580595
3,CA,-0.202277,0.396613,0.287496,0.156293,0.433135,-0.258746,-0.086195,-0.299612,-0.009857,-0.153508,1.688985,-0.216039,-0.486125,-0.550341
4,CO,-0.384493,-0.175867,0.286232,-0.271564,-0.123176,0.360960,-0.051274,0.349348,0.353040,-0.449836,-0.876665,-0.385810,-0.247019,-0.272589


We examined the summary descriptions:

In [0]:
df5B.describe()

,YTD Ret,Cash/AT,Cash/CL,Cash/INTX,CA/CL,LTDebt/AT,EBIT/INTX,PPE/AT,Sales/Employees,Confirmed,population,Death,Death_1M,Confirmd_1M
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,-0.347055,-0.246776,-0.196966,-0.093414,-0.213460,-0.031680,0.136177,0.226673,-0.102851,-0.375578,-0.755139,-0.319514,-0.255324,-0.286518
std,0.100442,0.427047,0.473554,0.256039,0.358243,0.622639,0.402473,0.568276,0.294946,0.546023,0.554586,0.527761,0.611539,0.640067
min,-0.619355,-0.862628,-0.723809,-0.295982,-0.725919,-1.154029,-0.521899,-0.824749,-0.655549,-0.556431,-1.085008,-0.469684,-0.562973,-0.635477
25%,-0.377266,-0.569995,-0.482496,-0.275149,-0.488190,-0.263319,-0.121703,-0.126053,-0.310604,-0.556431,-1.085008,-0.469684,-0.538111,-0.612120
50%,-0.335699,-0.286914,-0.222457,-0.184944,-0.274074,-0.029276,0.036147,0.148332,-0.112526,-0.526411,-0.951365,-0.447452,-0.470686,-0.514124
75%,-0.307277,-0.023340,-0.014705,-0.007466,-0.082584,0.256630,0.294207,0.498351,0.069198,-0.400448,-0.718082,-0.380757,-0.278570,-0.288476
max,-0.076600,1.189578,2.103964,0.853519,0.932315,1.953648,1.373128,2.258016,0.620483,3.109726,1.688985,3.148040,2.943445,2.704800


## Regression

In [0]:
import statsmodels.api as sm

X_s = df5B.iloc[:,2:15]
Y_s = df5B['YTD Ret']
X_s = sm.add_constant(X_s) # adding a constant
model_s = sm.OLS(Y_s, X_s).fit()
print_model_s = model_s.summary()
print(print_model_s)

                            OLS Regression Results                            
Dep. Variable:                YTD Ret   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     1.301
Date:                Sat, 18 Apr 2020   Prob (F-statistic):              0.260
Time:                        20:05:13   Log-Likelihood:                 52.398
No. Observations:                  48   AIC:                            -76.80
Df Residuals:                      34   BIC:                            -50.60
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3212      0.025    -

### Explanation

The State-level regression result does not change, and the adjusted R-Squared remains the same (7.7%). However, none of the variables are significant, which implies that our model does not fit the data. It is possible that this State-level regression and comparion cannot give many insights, especially our Y is generated on individual company performance. Although local governments announced different policies, the stock return varies by company's size, industry, actions in the shock, and history performances. If without a measurable collected on State-level, it is difficult to gain insights from the model.

# Graphical Analysis

We also visualized the ratios and COVID-19 data on Tableau. We exported the graphs into a pdf documents. For the reading convinience, we placed the explanation along with the chart inside of the pdf document. Please refer to **'Graphical Analysis.pdf'** to see our work.

Note: our conclusion is included in this Colab file.

# Conslusion

Although our regressions do not have much explanatory power (expected high adjusted R-Squared), we still gained some insights from the results and graphs. Some of the ratio coefficients can be understood and explained by the COVID-19 shock across the market, whereas the effect might not came from correlation. Further, adding COVID-19 variables into regression did not make a significant change, but improved the adjusted R-Squared. With the actual number of confirmed and death cases and the population, we were able to improve the adjusted R-Squared more. However, population was the only significant variable among the additional variables. 

Based on the graphs, we could not observe the same results as the regression output in most of the time. One reason is that the regression results are based on company-level data, and the coefficients are even small, while the graphs are constructed on market-sector level. But we can still notice that health care industry earns relatively high (and positive) returns. 

In conclusion, the combined results of regression and graphs show that the COVID-19 numbers cannot explain the volatility of the earning for individual companies. Other factors and effects from the COVID-19 shock might explain the variation better. On the high-level, we do observed that the entire market was **negatively influenced** by COVID-19 (negative `YTD Ret` on average). 